## All Offers Weekly Heatmap

In [1]:
import itertools
import datetime as dt
import psycopg2
import pandas as pd
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go

from nbstyler import DATA_STYLE as DS

plotly.offline.init_notebook_mode(connected=True)

%matplotlib notebook
%matplotlib inline

### Objectives



### Data Preparation

In [2]:
with psycopg2.connect('dbname=jobsbg') as conn:
    weekly_df = pd.read_sql_query('SELECT * FROM v_all_offers_count_weekly', conn, index_col='week_ts')

weekly_df.index = pd.to_datetime(weekly_df.index)
weekly_df.head(5)

,subm_count
week_ts,
2017-09-25,5287
2017-10-02,10281
2017-10-09,9501
2017-10-16,9672
2017-10-23,9303


Adjust week numbers and prepare a dictionary holding the data for each year. 

To adjust the week numbers for each year I used the https://www.epochconverter.com website.

In [3]:
yearly_dfs = {
    '2017': weekly_df[(weekly_df.index >= '2017-01-01') & (weekly_df.index <= '2017-12-31')],
    '2018': weekly_df[(weekly_df.index >= '2018-01-01') & (weekly_df.index <= '2018-12-30')],
    '2019': weekly_df[(weekly_df.index >= '2018-12-31') & (weekly_df.index <= '2019-12-29')],
    '2020': weekly_df[(weekly_df.index >= '2019-12-30') & (weekly_df.index <= '2020-03-28')],
}

#### Building the heatmap

The heatmap is composed of three dimensions: x, y, and z. The z dimension will hold our values matrix. Let's prepare that first:

In [4]:
cell_values = [
    [v for v in yearly_dfs['2020'].subm_count],
    [v for v in yearly_dfs['2019'].subm_count],
    [v for v in yearly_dfs['2018'].subm_count],
]

A manually defined color scale to match with the presentation style:

In [5]:
colorscale =[
    [0.0, DS['colorramp']['acc1'][0]],
    [0.1111111111111111, DS['colorramp']['acc1'][1]],
    [0.2222222222222222, DS['colorramp']['acc1'][2]],
    [0.3333333333333333, DS['colorramp']['acc1'][3]],
    [0.4444444444444444, DS['colorramp']['acc1'][4]],
    [0.5555555555555556, DS['colorramp']['acc1'][5]],
    [0.6666666666666666, DS['colorramp']['acc1'][6]],
    [0.7777777777777778, DS['colorramp']['acc1'][7]],
    [0.8888888888888888, DS['colorramp']['acc1'][8]],
    [1.0, DS['colorramp']['acc1'][9]]
]

Finally, the heatmap trace definition:

In [6]:
hm_trace = go.Heatmap(
    x=[n for n in range(1, 53)],
    y=['2020', '2019', '2018'],
    z=cell_values,
    hoverinfo='text+z+y',
    colorscale=colorscale,
    showscale=False,
    xgap=0,
    ygap=0,
)

data = [hm_trace]

Layout preparation is fairly standard. Labels have been turned off for both axes.

In [7]:
layout = go.Layout(
    paper_bgcolor=DS['colors']['bg1'],
    plot_bgcolor=DS['colors']['bg1'],
    title='Job Offer Submission Weekly Heatmap',
    titlefont=DS['chart_fonts']['title'],
    font=DS['chart_fonts']['text'],
    autosize=True,
    showlegend=False,
    hidesources=True,
    xaxis_title='Week',
    yaxis_title='Year',
    yaxis=dict(
        type='category',
    ),
)

In [8]:
fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename='data_jobs_weekly_subm_heatmap.html')

In [9]:
# Uncomment the line below to export an HTML version of the chart.
plotly.offline.plot(fig, filename='all_offers_weekly_subm_heatmap.html', show_link=False)

'all_offers_weekly_subm_heatmap.html'

In [10]:
from IPython.core.display import HTML
with open('../resources/styles/datum.css', 'r') as f:
    style = f.read()
HTML(style)